In [23]:
import pandas as pd
import functions as func
import numpy as np
import seaborn as sns
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, root_mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from collections import Counter


# load data sets
db_locations = func.import_yaml()


In [24]:
# load formated DS
df = pd.read_csv(db_locations['data_clean']['spacy_train'])
df_test = pd.read_csv(db_locations['data_clean']['spacy_test'])

In [25]:
df.head()

,PhraseId,SentenceId,Phrase,Sentiment,cleaned_text
0,1,1,A series of escapades demonstrating the adage ...,1,series escapade demonstrate adage good goose g...
1,2,1,A series of escapades demonstrating the adage ...,2,series escapade demonstrate adage good goose
2,3,1,A series,2,series
3,4,1,A,2,NaN
4,5,1,series,2,series


In [26]:
df.isnull().value_counts()

PhraseId  SentenceId  Phrase  Sentiment  cleaned_text
False     False       False   False      False           153384
                                         True              2676
Name: count, dtype: int64

In [27]:
df_test.head()

,PhraseId,SentenceId,Phrase,cleaned_text
0,156061,8545,An intermittently pleasing but mostly routine ...,intermittently pleasing routine effort
1,156062,8545,An intermittently pleasing but mostly routine ...,intermittently pleasing routine effort
2,156063,8545,An,NaN
3,156064,8545,intermittently pleasing but mostly routine effort,intermittently pleasing routine effort
4,156065,8545,intermittently pleasing but mostly routine,intermittently pleasing routine


In [28]:
df[df.cleaned_text.isnull()]

,PhraseId,SentenceId,Phrase,Sentiment,cleaned_text
3,4,1,A,2,NaN
6,7,1,of,2,NaN
13,14,1,the,2,NaN
16,17,1,that,2,NaN
18,19,1,what,2,NaN
...,...,...,...,...,...
155587,155588,8515,a few others,2,NaN
155588,155589,8515,few others,2,NaN
155591,155592,8515,do n't make often enough,2,NaN
155592,155593,8515,make often enough,2,NaN


In [29]:
df[df.cleaned_text.isnull()& df.Sentiment == 1].sort_values('Phrase').duplicated().sum()

0

In [30]:
df_test.isnull().value_counts()

PhraseId  SentenceId  Phrase  cleaned_text
False     False       False   False           64829
                              True             1462
                      True    True                1
Name: count, dtype: int64

In [31]:
df_test[df_test.cleaned_text.isnull()]

,PhraseId,SentenceId,Phrase,cleaned_text
2,156063,8545,An,NaN
9,156070,8545,but,NaN
11,156072,8545,mostly,NaN
14,156075,8545,.,NaN
19,156080,8546,is really,NaN
...,...,...,...,...
66126,222187,11844,not quite enough,NaN
66127,222188,11844,quite enough,NaN
66216,222277,11850,-- but,NaN
66248,222309,11852,even if you,NaN


In [32]:
#Droping nan columns for the first test

df.dropna(inplace = True)
df_test.fillna("", inplace=True)

In [33]:
#Testing what we learned in class

# Crop the sample size because the fowloing tests needed to much memory for current tests

frac_df = df.sample(frac = 0.2)

# Data preparation with TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=5000) 

X_train = tfidf.fit_transform(frac_df['cleaned_text']).toarray()
X_test = tfidf.transform(df_test['cleaned_text']).toarray()

# Target variable (Sentiment)
y_train = frac_df['Sentiment']

# Split the data into training and validation sets
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [34]:

# Balance the data frame with SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train, y_train)

In [13]:
# Train a Logistic Regression model
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train_split, y_train_split)

# Make predictions on the validation set
y_pred = model.predict(X_val_split)

/home/tiago/Documents/GitHub/Projects/final_project/proj_env/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [14]:
# Evaluate the model and Report
#func.print_evaluation(y_val_split , y_pred,X_train_split, y_train_split,model)

print(classification_report(y_val_split, y_pred))
print(f'Validation Accuracy: {accuracy_score(y_val_split, y_pred): .2f}')
print(f"MAE {mean_absolute_error(y_pred, y_val_split): .2f}") 
print(f"RMSE, {root_mean_squared_error(y_pred, y_val_split): .2f}")
print(f"R2 score, {model.score(X_train_split, y_train_split): .2f}")

              precision    recall  f1-score   support

           0       0.58      0.11      0.18       274
           1       0.46      0.24      0.32      1081
           2       0.60      0.88      0.72      3101
           3       0.49      0.34      0.40      1310
           4       0.47      0.11      0.18       370

    accuracy                           0.57      6136
   macro avg       0.52      0.34      0.36      6136
weighted avg       0.55      0.57      0.52      6136

Validation Accuracy:  0.57
MAE  0.49
RMSE,  0.80
R2 score,  0.66


In [15]:
y_pred

array([3, 3, 2, ..., 3, 2, 2])

In [16]:
# Train a Logistic KNN classifier model

from sklearn.neighbors import KNeighborsClassifier

# Train a KNN classifier model
model = KNeighborsClassifier(n_neighbors=3) 
model.fit(X_train_split,y_train_split)

# Make predictions on the validation set
y_pred = model.predict(X_val_split)


In [17]:
# Evaluate the model and Report
#func.print_evaluation(y_val_split , y_pred,X_train_split, y_train_split,model)

print(classification_report(y_val_split, y_pred))
print(f'Validation Accuracy: {accuracy_score(y_val_split, y_pred): .2f}')
print(f"MAE {mean_absolute_error(y_pred, y_val_split): .2f}") 
print(f"RMSE, {root_mean_squared_error(y_pred, y_val_split): .2f}")
print(f"R2 score, {model.score(X_train_split, y_train_split): .2f}")

              precision    recall  f1-score   support

           0       0.28      0.26      0.27       274
           1       0.33      0.25      0.29      1081
           2       0.58      0.80      0.67      3101
           3       0.47      0.23      0.31      1310
           4       0.48      0.15      0.23       370

    accuracy                           0.52      6136
   macro avg       0.43      0.34      0.35      6136
weighted avg       0.49      0.52      0.48      6136

Validation Accuracy:  0.52
MAE  0.59
RMSE,  0.90
R2 score,  0.71


In [18]:
y_pred

array([2, 2, 1, ..., 2, 2, 2])

In [19]:
# Train a AdaBoost model
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor

# Train a Logistic Regression model
model = GradientBoostingRegressor(max_depth=5,
                                   n_estimators=10)

model.fit(X_train_split,y_train_split)

# Make predictions on the validation set
y_pred = model.predict(X_val_split)



In [20]:
y_pred

array([2.27985071, 2.05365947, 2.05365947, ..., 2.28138816, 2.05365947,
       2.05365947])

In [21]:
# Evaluate the model and Report
#func.print_evaluation(y_val_split , y_pred,X_train_split, y_train_split,model)

#print(classification_report(y_val_split, y_pred))
#print(f'Validation Accuracy: {accuracy_score(y_val_split, y_pred): .2f}')
print(f"MAE {mean_absolute_error(y_pred, y_val_split): .2f}") 
print(f"RMSE, {root_mean_squared_error(y_pred, y_val_split): .2f}")
print(f"R2 score, {model.score(X_train_split, y_train_split): .2f}")

MAE  0.61
RMSE,  0.88
R2 score,  0.05


In [22]:
# Train a Decision Tree model
from sklearn.tree import DecisionTreeRegressor

model =   DecisionTreeRegressor(max_depth=2)
model.fit(X_train_split,y_train_split)

#Make predictions on the validation set
y_pred = model.predict(X_val_split)


In [23]:
y_pred

array([2.65632458, 2.0601141 , 2.0601141 , ..., 2.0601141 , 2.0601141 ,
       2.0601141 ])

In [24]:
# Evaluate the model and Report
#func.print_evaluation(y_val_split , y_pred,X_train_split, y_train_split,model)

#print(classification_report(y_val_split, y_pred))
#print(f'Validation Accuracy: {accuracy_score(y_val_split, y_pred): .2f}')
print(f"MAE {mean_absolute_error(y_pred, y_val_split): .2f}") 
print(f"RMSE, {root_mean_squared_error(y_pred, y_val_split): .2f}")
print(f"R2 score, {model.score(X_train_split, y_train_split): .2f}")

MAE  0.62
RMSE,  0.89
R2 score,  0.03


In [25]:
# Train a  Support Vector Machine
from sklearn.svm import SVC

model = SVC(kernel='linear')
model.fit(X_train, y_train)

# Make predictions on the validation set
y_pred = model.predict(X_val_split)


In [26]:
y_pred

array([2, 2, 2, ..., 3, 2, 1])

In [27]:
# Evaluate the model and Report
#func.print_evaluation(y_val_split , y_pred,X_train_split, y_train_split,model)

print(classification_report(y_val_split, y_pred))
print(f'Validation Accuracy: {accuracy_score(y_val_split, y_pred): .2f}')
print(f"MAE {mean_absolute_error(y_pred, y_val_split): .2f}") 
print(f"RMSE, {root_mean_squared_error(y_pred, y_val_split): .2f}")
print(f"R2 score, {model.score(X_train_split, y_train_split): .2f}")

              precision    recall  f1-score   support

           0       0.66      0.24      0.35       274
           1       0.61      0.38      0.47      1081
           2       0.66      0.91      0.77      3101
           3       0.61      0.48      0.54      1310
           4       0.80      0.23      0.36       370

    accuracy                           0.65      6136
   macro avg       0.67      0.45      0.50      6136
weighted avg       0.65      0.65      0.62      6136

Validation Accuracy:  0.65
MAE  0.40
RMSE,  0.72
R2 score,  0.66


In [28]:
# Train a Bagging and Pasting model
from xgboost import XGBClassifier

# Train a Logistic Regression model
model = XGBClassifier()

model.fit(X_train_split,y_train_split)

# Make predictions on the validation set
y_pred = model.predict(X_val_split)


In [29]:
y_pred

array([2, 2, 2, ..., 2, 2, 2])

In [30]:
# Evaluate the model and Report
func.print_evaluation(y_val_split , y_pred,X_train_split, y_train_split,model)

              precision    recall  f1-score   support

           0       0.51      0.15      0.23       274
           1       0.51      0.12      0.19      1081
           2       0.57      0.95      0.71      3101
           3       0.47      0.18      0.26      1310
           4       0.47      0.13      0.21       370

    accuracy                           0.55      6136
   macro avg       0.50      0.31      0.32      6136
weighted avg       0.53      0.55      0.47      6136

Validation Accuracy:  0.55
MAE  0.53
RMSE,  0.84
R2 score,  0.63


In [31]:
# Train a Bagging and Pasting model
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor

# Train a Logistic Regression model
model = BaggingRegressor(DecisionTreeRegressor(max_depth=5),
                               n_estimators=10, 
                               max_samples = 10)

model.fit(X_train_split,y_train_split)

# Make predictions on the validation set
y_pred = model.predict(X_val_split)


In [32]:
# Evaluate the model and Report
#func.print_evaluation(y_val_split , y_pred,X_train_split, y_train_split,model)

#print(classification_report(y_val_split, y_pred))
#print(f'Validation Accuracy: {accuracy_score(y_val_split, y_pred): .2f}')
print(f"MAE {mean_absolute_error(y_pred, y_val_split): .2f}") 
print(f"RMSE, {root_mean_squared_error(y_pred, y_val_split): .2f}")
print(f"R2 score, {model.score(X_train_split, y_train_split): .2f}")

MAE  0.67
RMSE,  0.90
R2 score, -0.01


In [33]:
y_pred

array([2.16, 2.16, 2.16, ..., 2.16, 2.16, 2.16])

In [43]:
# Train a Random Patches model
from sklearn.ensemble import RandomForestRegressor

# Train a Logistic Regression model
model = RandomForestRegressor(n_estimators=10,
                             max_depth=5)

model.fit(X_train_split,y_train_split)

# Make predictions on the validation set
y_pred = model.predict(X_val_split)

In [44]:
y_pred

array([2.55396286, 2.04993103, 2.04993103, ..., 2.16519518, 2.04993103,
       2.04993103])

In [46]:
# Evaluate the model and Report
#func.print_evaluation(y_val_split , y_pred,X_train_split, y_train_split,model)

#print(classification_report(y_val_split, y_pred))
#print(f'Validation Accuracy: {accuracy_score(y_val_split, y_pred): .2f}')
print(f"MAE {mean_absolute_error(y_pred, y_val_split): .2f}") 
print(f"RMSE, {root_mean_squared_error(y_pred, y_val_split): .2f}")
print(f"R2 score, {model.score(X_train_split, y_train_split): .2f}")

MAE  0.61
RMSE,  0.88
R2 score,  0.06


In [37]:
# Using grid search for decision tree
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor


# Setting parameters 

grid = {"max_leaf_nodes": [5, 25,50],
        "max_depth":[2,4,8]}

dt = DecisionTreeRegressor()

# Train a Logistic Regression model

model = GridSearchCV(estimator = dt, param_grid = grid, cv=5, verbose=10) # The "cv" option here is used to provide the desired number of folds K.
model.fit(X_train_split,y_train_split)


# Make predictions on the validation set
y_pred = model.predict(X_val_split)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START max_depth=2, max_leaf_nodes=5...............................
[CV 1/5; 1/9] END max_depth=2, max_leaf_nodes=5;, score=0.018 total time=   1.8s
[CV 2/5; 1/9] START max_depth=2, max_leaf_nodes=5...............................
[CV 2/5; 1/9] END max_depth=2, max_leaf_nodes=5;, score=0.028 total time=   1.8s
[CV 3/5; 1/9] START max_depth=2, max_leaf_nodes=5...............................
[CV 3/5; 1/9] END max_depth=2, max_leaf_nodes=5;, score=0.024 total time=   1.8s
[CV 4/5; 1/9] START max_depth=2, max_leaf_nodes=5...............................
[CV 4/5; 1/9] END max_depth=2, max_leaf_nodes=5;, score=0.016 total time=   1.8s
[CV 5/5; 1/9] START max_depth=2, max_leaf_nodes=5...............................
[CV 5/5; 1/9] END max_depth=2, max_leaf_nodes=5;, score=0.023 total time=   1.8s
[CV 1/5; 2/9] START max_depth=2, max_leaf_nodes=25..............................
[CV 1/5; 2/9] END max_depth=2, max_leaf_nodes=25;

In [38]:
y_pred

array([2.52763819, 2.04649157, 2.04649157, ..., 2.04649157, 2.04649157,
       2.04649157])

In [39]:

# Evaluate the model
print(f'The best model was {model.best_estimator_}')

# Evaluate the model and Report
#func.print_evaluation(y_val_split , y_pred,X_train_split, y_train_split,model)

#print(classification_report(y_val_split, y_pred))
#print(f'Validation Accuracy: {accuracy_score(y_val_split, y_pred): .2f}')
print(f"MAE {mean_absolute_error(y_pred, y_val_split): .2f}") 
print(f"RMSE, {root_mean_squared_error(y_pred, y_val_split): .2f}")
print(f"R2 score, {model.score(X_train_split, y_train_split): .2f}")

The best model was DecisionTreeRegressor(max_depth=8, max_leaf_nodes=25)
MAE  0.60
RMSE,  0.87
R2 score,  0.06


In [40]:
# Using Random Search for decision tree
from sklearn.model_selection import GridSearchCV

# Setting parameters 

grid = {"max_leaf_nodes": [int(x) for x in np.linspace(start = 5, stop = 30, num = 3)],
        "max_depth":[int(x) for x in np.linspace(1, 11, num = 3)]}

dt = DecisionTreeRegressor()

# Train a Logistic Regression model

model = GridSearchCV(estimator = dt, param_grid = grid, cv=5, verbose=10) # The "cv" option here is used to provide the desired number of folds K.
model.fit(X_train_split,y_train_split)


# Make predictions on the validation set
y_pred = model.predict(X_val_split)


Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START max_depth=1, max_leaf_nodes=5...............................
[CV 1/5; 1/9] END max_depth=1, max_leaf_nodes=5;, score=0.016 total time=   1.0s
[CV 2/5; 1/9] START max_depth=1, max_leaf_nodes=5...............................
[CV 2/5; 1/9] END max_depth=1, max_leaf_nodes=5;, score=0.019 total time=   1.0s
[CV 3/5; 1/9] START max_depth=1, max_leaf_nodes=5...............................
[CV 3/5; 1/9] END max_depth=1, max_leaf_nodes=5;, score=0.020 total time=   1.0s
[CV 4/5; 1/9] START max_depth=1, max_leaf_nodes=5...............................
[CV 4/5; 1/9] END max_depth=1, max_leaf_nodes=5;, score=0.010 total time=   1.0s
[CV 5/5; 1/9] START max_depth=1, max_leaf_nodes=5...............................
[CV 5/5; 1/9] END max_depth=1, max_leaf_nodes=5;, score=0.020 total time=   1.0s
[CV 1/5; 2/9] START max_depth=1, max_leaf_nodes=17..............................
[CV 1/5; 2/9] END max_depth=1, max_leaf_nodes=17;

In [41]:
y_pred

array([2.71078431, 2.04088173, 2.04088173, ..., 3.16      , 2.04088173,
       2.04088173])

In [42]:

# Evaluate the model
print(f'The best model was {model.best_estimator_}')

# Evaluate the model and Report
#func.print_evaluation(y_val_split , y_pred,X_train_split, y_train_split,model)

#print(classification_report(y_val_split, y_pred))
#print(f'Validation Accuracy: {accuracy_score(y_val_split, y_pred): .2f}')
print(f"MAE {mean_absolute_error(y_pred, y_val_split): .2f}") 
print(f"RMSE, {root_mean_squared_error(y_pred, y_val_split): .2f}")
print(f"R2 score, {model.score(X_train_split, y_train_split): .2f}")

The best model was DecisionTreeRegressor(max_depth=11, max_leaf_nodes=17)
MAE  0.60
RMSE,  0.87
R2 score,  0.07


In [ ]:
#  Hyperparameter Tuning for the best models

In [47]:
param_grid_lr = {
    'C': [0.01, 0.1, 1, 10],  # Regularization strength
    'penalty': ['l1', 'l2'],  # Type of regularization
    'solver': ['liblinear']  # liblinear is necessary for L1 regularization
}

log_reg = LogisticRegression(max_iter=500)

grid_search_lr = GridSearchCV(log_reg, param_grid_lr, cv=5, scoring='accuracy')
grid_search_lr.fit(X_train_split,y_train_split)

print("Best Parameters for Logistic Regression:", grid_search_lr.best_params_)

Best Parameters for Logistic Regression: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}


In [55]:
# using the bes parameters and check th score again
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

param_grid_lr = {
    'C': [10], 
    'penalty': ['l2'],  
    'solver': ['liblinear']  
}

log_reg = LogisticRegression(max_iter=500)
grid_search_lr = GridSearchCV(log_reg, param_grid_lr, cv=5, scoring='accuracy')
grid_search_lr.fit(X_train_split,y_train_split)
y_pred = grid_search_lr.predict(X_val_split)
func.print_evaluation(y_val_split , y_pred,X_train_split, y_train_split,grid_search_lr)


              precision    recall  f1-score   support

           0       0.34      0.18      0.23       280
           1       0.44      0.32      0.37      1103
           2       0.64      0.80      0.71      3069
           3       0.48      0.41      0.44      1307
           4       0.41      0.24      0.30       377

    accuracy                           0.57      6136
   macro avg       0.46      0.39      0.41      6136
weighted avg       0.54      0.57      0.54      6136

Validation Accuracy:  0.57
MAE  0.51
RMSE,  0.82
R2 score,  0.73


In [56]:
y_pred = grid_search_lr.predict(X_test)

In [57]:
df_test

,PhraseId,SentenceId,Phrase,cleaned_text
0,156061,8545,An intermittently pleasing but mostly routine ...,intermittently pleasing routine effort
1,156062,8545,An intermittently pleasing but mostly routine ...,intermittently pleasing routine effort
2,156063,8545,An,
3,156064,8545,intermittently pleasing but mostly routine effort,intermittently pleasing routine effort
4,156065,8545,intermittently pleasing but mostly routine,intermittently pleasing routine
...,...,...,...,...
66287,222348,11855,"A long-winded , predictable scenario .",long wind predictable scenario
66288,222349,11855,"A long-winded , predictable scenario",long wind predictable scenario
66289,222350,11855,"A long-winded ,",long winded
66290,222351,11855,A long-winded,long winded


In [ ]:
param_grid_svm = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1],
    'kernel': ['linear', 'rbf', 'poly']  # Different kernel types
}

svm = SVC()

grid_search_svm = GridSearchCV(svm, param_grid_svm, cv=5, scoring='accuracy')
grid_search_svm.fit(X_train_split,y_train_split)

print("Best Parameters for SVM:", grid_search_svm.best_params_)

In [58]:
df_test.PhraseId

0        156061
1        156062
2        156063
3        156064
4        156065
          ...  
66287    222348
66288    222349
66289    222350
66290    222351
66291    222352
Name: PhraseId, Length: 66292, dtype: int64

In [60]:

submission = pd.DataFrame({
    'PhraseId': df_test.PhraseId,  # This should be the ID column from the test data
    'Sentiment': y_pred  # Your prediction array
})


submission.head()

,PhraseId,Sentiment
0,156061,3
1,156062,3
2,156063,2
3,156064,3
4,156065,3


In [61]:
submission.to_csv('../data/clean/submission', index=False)  # Save the file without the index
